In [1]:
import pandas as pd
import json
import spacy
import random
from spacy.tokens import Doc
from spacy.training import Example
from spacy.util import minibatch, compounding
import en_core_web_lg
import warnings

## Recipe text extract

In [2]:
recipes = pd.read_csv (r'RAW_recipes.csv')
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [3]:
recipes = recipes.loc[recipes['n_ingredients'] < 6]
recipes = recipes.loc[recipes['n_steps'] == 5]

In [4]:
step = recipes['steps']
step_final = []
for step_txt in step:
    step_txt = step_txt.replace("[", "")
    step_txt = step_txt.replace("]", "")
    step_txt = step_txt.replace("', '", ". ")
    step_txt = step_txt.replace("\' " , " ")
    step_final.append(step_txt)

In [5]:
df = pd.DataFrame(step_final, columns =['steps'])
df

,steps
0,'mix the herbs and vinegar together in a jar. ...
1,"'for one regular size bunch of grapes , 2 egg ..."
2,"'soak peas overnight , drain and add 3 quarts ..."
3,'toast the bread slices. spread one side of ea...
4,'mix ingredients together and add to a clean s...
...,...
4958,'julienne just the outsides of each zucchini l...
4959,'bring juices and sugar to a boil. add squash ...
4960,"'in a saucepan , bring the broth to a simmer o..."
4961,'strip and wash sorrel. cook in a saucepan wit...


In [6]:
text = '\n'.join(df['steps'])
text = text.replace("[", "")
text = text.replace("]", "")
text = text.replace("', '", ". ")
text = text.replace("\' " , " ")

In [7]:
print(text)

'mix the herbs and vinegar together in a jar. seal and leave in a warm place for 2 weeks , shaking often. strain the vinegar through cheesecloth and then through coffee filter paper. pour into sterilised bottles and add 2-3 sprigs of fresh herb to each bottle. seal with acid-proof lids or bottle tops'
'for one regular size bunch of grapes , 2 egg whites. brush grapes with egg whites and then sprinkle sugar to totaly cover grapes. freeze until needed the day of', "this makes a very pretty garnish that's really yummmy !", 'the crystallized sugar makes the grapes crunchy and sweet !'
'soak peas overnight , drain and add 3 quarts water. bring to a boil and cook rapidly. skim off any skins. after cooking for an hour , add pork and simmer and simmer for 2 hours or until pork is tender. add seasonings'
'toast the bread slices. spread one side of each bread slice very lightly with butter. put cheese slice between buttered sides of bread. heat in the microwave until cheese starts to melt. in my

In [8]:
import os
from pathlib import Path

root_location = Path(r"C:\Users\gghan\OneDrive\Desktop")
os.makedirs(root_location, exist_ok=True)
file = 'text.txt'



with open(root_location / file.format, "w") as f:
    f.write(str(text))

TypeError: unsupported operand type(s) for /: 'WindowsPath' and 'builtin_function_or_method'

In [48]:
import os
from pathlib import Path

root_location = Path(r"C:\Users\gghan\OneDrive\Desktop\recipes")
os.makedirs(root_location, exist_ok=True)
file = 'text{}.txt'
n = 0

for index, row in df.iterrows():
    with open(root_location / file.format(n), "w") as f:
        f.write(str(row['steps']))
        n+=1

## Custom NER Model

In [4]:
with open ('annotations.json') as f:
    training_data = json.load(f)

In [5]:
training_data.pop('classes')

['FOOD']

In [6]:
# TRAIN_DATA = []
# for text, annotations in training_data['annotations']:
#     for annotation in annotations['entities']: 
#         print(annotation)

In [7]:
# TRAIN_DATA

In [8]:
# def test_model(model, text):
#     doc = nlp(text)
#     results = []
#     entities = []
#     for ent in doc.ents:
#         entities.append((ent.start_char, ent.end_char, ent.label_))
#     if len(entities) > 0:
#         results = [text, {"entities": entities}]
#         return (results)

In [9]:
# for text, annotations in training_data['annotations']:
#     print(annotations)

In [10]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print (losses)
    return (nlp)

In [ ]:
# add NER to the pipeline and the new label
nlp = en_core_web_lg.load()
ner = nlp.get_pipe("ner")
ner.add_label("FOOD")

# get the names of the components we want to disable during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# start the training loop, only training NER
epochs = 30
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    sizes = compounding(1.0, 4.0, 1.001)
    
    # batch up the examples using spaCy's minibatc
    for epoch in range(epochs):
        examples = training_data['annotations']
        batches = minibatch(examples, size=sizes)
        losses = {}
        
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)

        print("Losses ({}/{})".format(epoch + 1, epochs), losses)

In [ ]:
# nlp = train_spacy(TRAIN_DATA, 30)
# nlp.to_disk("ner_model")